In [2]:
import sys
import os
sys.path.append('site-packages') # manually put all packages/libraries into this folder
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras import models
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from scene_cluster_io import *

In [6]:
film = 'extremely_wicked'
frame_choice = list(range(650, 1050))
threshold = 3000

dialogue_folder = os.path.join('dialogue_frames', film)
print('There are', len(os.listdir(dialogue_folder)), 'images in the folder')
print('Selected', len(frame_choice), 'of those frames')

hac_labels = label_clusters(dialogue_folder, frame_choice, film, threshold)

There are 6603 images in the folder
Selected 400 of those frames
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [7]:
tuned_model = models.load_model('saved_models/tuned_model')

In [8]:
y_pred_values = predict_mcu(dialogue_folder, tuned_model, frame_choice, film)

In [9]:
shot_id_list = get_shot_ids(frame_choice, hac_labels)

In [10]:
scene_df = pd.DataFrame(zip(frame_choice, hac_labels, shot_id_list, y_pred_values), columns=['frame_file', 'cluster', 'shot_id', 'mcu'])
scene_df.head(3)

,frame_file,cluster,shot_id,mcu
0,650,2,0,1
1,651,2,0,1
2,652,2,0,1


In [11]:
alternating_pairs = get_alternating_pairs(frame_choice, hac_labels, y_pred_values, shot_id_list)
alternating_pairs

[[0, 2], [9, 13], [0, 1], [5, 10], [0, 6]]

In [12]:
speaker_pairs = mcu_check(alternating_pairs, scene_df)
speaker_pairs

cluster	 count	 mcu probability
0 	 165 	 48.48%
2 	 61 	 65.57%
Fails MCU check

9 	 28 	 96.43%
13 	 21 	 100.00%
Passes MCU check

0 	 165 	 48.48%
1 	 9 	 66.67%
Fails MCU check

5 	 23 	 95.65%
10 	 17 	 100.00%
Passes MCU check

0 	 165 	 48.48%
6 	 9 	 55.56%
Fails MCU check



[[9, 13], [5, 10]]

In [13]:
anchors = anchor_scenes(speaker_pairs, scene_df)
anchors

Speaker A and B Clusters: [9, 13]
Anchor Start/End Frames: 690 743

Speaker A and B Clusters: [5, 10]
Anchor Start/End Frames: 954 997



[(690, 743), (954, 997)]

In [14]:
scenes = expand_scenes(speaker_pairs, scene_df)
scenes

Speaker A and B Clusters: [9, 13]
Anchor Start/End Frames: 690 743
Cutaway Clusters: [0]
Expanded Start/End Frames: 690 745

Speaker A and B Clusters: [5, 10]
Anchor Start/End Frames: 954 997
Cutaway Clusters: [16]
Expanded Start/End Frames: 954 997



[(690, 745), (954, 997)]

All at once

In [16]:
film = 'hustle'
frame_choice = list(range(600, 1000))
threshold = 3000

dialogue_folder = os.path.join('dialogue_frames', film)
print('There are', len(os.listdir(dialogue_folder)), 'images in the folder')
print('Selected', len(frame_choice), 'of those frames')

hac_labels = label_clusters(dialogue_folder, frame_choice, film, threshold)
y_pred_values = predict_mcu(dialogue_folder, tuned_model, frame_choice, film)
shot_id_list = get_shot_ids(frame_choice, hac_labels)
scene_df = pd.DataFrame(zip(frame_choice, hac_labels, shot_id_list, y_pred_values), columns=['frame_file', 'cluster', 'shot_id', 'mcu'])
alternating_pairs = get_alternating_pairs(frame_choice, hac_labels, y_pred_values, shot_id_list)
speaker_pairs = mcu_check(alternating_pairs, scene_df)
anchors = anchor_scenes(speaker_pairs, scene_df)
scenes = expand_scenes(speaker_pairs, scene_df)

There are 5877 images in the folder
Selected 400 of those frames
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [18]:
film = 'hustle'
frame_choice = list(range(600, 1000))
threshold = 3000

scene_list = []

In [17]:
def generate_scenes(film, frame_choice, threshold):
    dialogue_folder = os.path.join('dialogue_frames', film)
    print('There are', len(os.listdir(dialogue_folder)), 'images in the folder')
    print('Selected', len(frame_choice), 'of those frames')

    hac_labels = label_clusters(dialogue_folder, frame_choice, film, threshold)
    y_pred_values = predict_mcu(dialogue_folder, tuned_model, frame_choice, film)
    shot_id_list = get_shot_ids(frame_choice, hac_labels)
    scene_df = pd.DataFrame(zip(frame_choice, hac_labels, shot_id_list, y_pred_values), columns=['frame_file', 'cluster', 'shot_id', 'mcu'])
    alternating_pairs = get_alternating_pairs(frame_choice, hac_labels, y_pred_values, shot_id_list)
    speaker_pairs = mcu_check(alternating_pairs, scene_df)
    anchors = anchor_scenes(speaker_pairs, scene_df)
    scenes = expand_scenes(speaker_pairs, scene_df)

    return scenes

In [25]:
a = 500
b = a - 200
c = a + 200

list(range(b, c))

[300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 462,
 463,
 464,
 465,
 466

In [32]:
film = 'hustle'
threshold = 3000
frame_choices = [list(range(600, 1000)), list(range(800, 1200)), list(range(1000, 1400))]
scene_list = []

for frames in frame_choices:
    scene_list.append(generate_scenes(film, frames, threshold))

There are 5877 images in the folder
Selected 400 of those frames
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

Number of clusters: 49
cluster	 count	 mcu probability
0 	 12 	 41.67%
4 	 33 	 63.64%
Fails MCU check

4 	 33 	 63.64%
10 	 12 	 91.67%
Passes MCU check

Speaker A and B Clusters: [4, 10]
Anchor Start/End Frames: 1055 1399

Speaker A and B Clusters: [4, 10]
Anchor Start/End Frames: 1055 1399
Cutaway Clusters: [ 0 34 46 19 17  1  7 13 18 38 37 35 43 28 26 30 42 24 11 29 36  8 16 14
 48 25 45 21 44  9 12 31  6 15  3 40 39 41]
Expanded Start/End Frames: 1054 1399



In [33]:
scene_list

[[(648, 760), (814, 917)], [(814, 917)], [(1054, 1399)]]